In [ ]:
import threading
import random
import time
import math
import sys
import os

from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

%matplotlib inline

sys.path.append(os.path.abspath('..'))

In [ ]:
from mule.low_control import MuleController
from mule.bridge import TCPMule

controller = MuleController()

mule = TCPMule(controller.update)

def update_function():
    for timestep, _ in enumerate(mule.run()):
        controller.update(mule)

t = threading.Thread(target=update_function)
t.start()

In [ ]:
neighbourhood_size = 5
step_size = 2.5

In [ ]:
# Utilities

def collides_with_obstacle(obstacle, x, y):
    x_dist = math.fabs(obstacle['center'][0] - x)
    y_dist = math.fabs(obstacle['center'][2] - y)
    
    x_collision = x_dist < (obstacle['extents'][0] + 0.8)
    y_collision = y_dist < (obstacle['extents'][2] + 0.8)

    return x_collision and y_collision

def distance(p1, p2):
    x_dist = p1[0] - p2[0]
    y_dist = p1[1] - p2[1]
    
    return math.sqrt(x_dist ** 2 + y_dist ** 2)

In [ ]:
# Sample a random point

def rrt_sample_free():
    random.seed()

    sample = None

    while sample is None:
        sample = (random.uniform(36, 63), random.uniform(35, 50))

        for detectedObject in mule.detectedObjects:
            if detectedObject['type'] != 'Blockers':
                continue

            if collides_with_obstacle(detectedObject, sample[0], sample[1]):
                sample = None
                break

    return sample

In [ ]:
# Maintain graph

def rrt_add_node(end, start):
    rrt_graph[end] = []
    rrt_graph[start].append(end)
    rrt_parent[end] = start
    rrt_cost[end] = rrt_cost[start] + distance(start, end)

def rrt_path(current):
    path = [current]

    while current in rrt_parent:        
        current = rrt_parent[current]

        path.append(current)

    return path

def rrt_nearest_neighbour(x_rand):
    min_dist = 100000
    
    closest = None

    for node in rrt_graph.keys():
        dist = distance(node, x_rand)
        
        if dist < min_dist:
            min_dist = dist
            closest = node
    
    return closest

def rrt_neighbourhood(x_rand):
    neighbourhood = []

    for node in rrt_graph:
        if distance(x_rand, node) <= neighbourhood_size:
            neighbourhood.append(node)

    return neighbourhood


In [ ]:
# Is the path collsion free

def rrt_obstacle_free(start, end):
    dx = end[0] - start[0]
    dy = end[1] - start[1]
    
    for detectedObject in mule.detectedObjects:
        if detectedObject['type'] != 'Blockers':
            continue

        for t in range(0, 11):
            x = start[0] + dx * t * 0.1
            y = start[1] + dy * t * 0.1

            if collides_with_obstacle(detectedObject, x, y):
                return False

    return True

# Steer start towards end
# TODO Incorporate vehicle dynamics into this.

def rrt_steer(start, end):
    return end

    if distance(start, end) < step_size:
        return end
    else:
        theta = math.atan2(start[0] - end[0], start[1] - end[1])
        x = start[0] + step_size * math.cos(theta)
        y = start[1] + step_size * math.sin(theta)
        return (x, y)

In [ ]:
# The RRT algorithm

def rrt_step():
    x_rand = rrt_sample_free()
    x_nearest = rrt_nearest_neighbour(x_rand)
    x_new = rrt_steer(x_nearest, x_rand)

    if rrt_obstacle_free(x_nearest, x_new):
        X_near = rrt_neighbourhood(x_new)

        x_min = x_nearest
        c_min = rrt_cost[x_nearest] + distance(x_nearest, x_new)

        for x_near in X_near:
            if rrt_obstacle_free(x_near, x_new) and (rrt_cost[x_near] + distance(x_near, x_new) < c_min):
                x_min = x_near
                c_min = rrt_cost[x_near] + distance(x_near, x_new)

        rrt_add_node(x_new, x_min)
    
        for x_near in X_near:
            if rrt_obstacle_free(x_near, x_new) and (rrt_cost[x_new] + distance(x_near, x_new) < rrt_cost[x_near]):
                rrt_parent[x_near] = x_new

In [ ]:
random.seed(time.time())

start = (mule.sensorData['position_x'], mule.sensorData['position_z'])
end = (63, 40)

rrt_graph = {start: [], end: []}
rrt_cost = {start: 0, end: distance(start, end) * 10}

rrt_parent = {end: start}

for _ in range(0, 200):
    rrt_step()

In [ ]:
fig, ax = plt.subplots(1, figsize=(20, 20))

for detectedObject in mule.detectedObjects:
    if detectedObject['type'] != 'Blockers':
        continue
    
    xy = (
        detectedObject['center'][0] - detectedObject['extents'][0],
        detectedObject['center'][2] - detectedObject['extents'][2],
    )
    
    width  = detectedObject['extents'][0] * 2
    height = detectedObject['extents'][2] * 2

    ax.add_patch(Rectangle(xy, width, height, linewidth=1,edgecolor='r',facecolor='none'))

mule_xy = (
    mule.sensorData['position_x'] - 0.5, 
    mule.sensorData['position_z'] - 1,
)

ax.add_patch(Rectangle(mule_xy, 1, 2, linewidth=1,edgecolor='g',facecolor='none'))

ax.set_aspect('equal')

for start, tos in rrt_graph.iteritems():
    for to in tos:
        plt.plot((start[0], to[0]), (start[1], to[1]), color='g')

path = rrt_path(end)
plt.plot([x[0] for x in path], [x[1] for x in path], color='r')

plt.show()

In [ ]:
waypoints = path[::-1]

waypoint = 0

controller.desired_velocity = 2.

print waypoints[waypoint]

while True:
    x_diff = waypoints[waypoint][0] - mule.sensorData['position_x']
    y_diff = waypoints[waypoint][1] - mule.sensorData['position_z']

    magnitude = math.sqrt(x_diff ** 2 + y_diff ** 2)

    controller.desired_phi = math.degrees(math.atan2(x_diff, y_diff))

    print magnitude

    if magnitude < 2:
        waypoint += 1

        print waypoints[waypoint]

        if waypoint == len(waypoints):
            mule.send_bus_req(mule.rightDriveInput, 0)
            mule.send_bus_req( mule.leftDriveInput, 0)

            mule.send_bus_req(mule.brakeInput, 1)

            mule.send_bus_req(mule.steerInput, 0)

            print "ARRIVED!"

            break

    time.sleep(0.05)
